```{contents}
```
## RunnableLambda 

**RunnableLambda** is a LangChain runnable that wraps a **Python callable (function or lambda)** so it can participate **natively in LCEL (LangChain Expression Language) pipelines**.

It allows you to apply **custom logic or lightweight transformations** inside a runnable chain.

Provided by LangChain.

```
Input
  ↓
RunnableLambda (custom logic)
  ↓
Output
```

---

### Why RunnableLambda Exists

In real pipelines, you often need:

* Small transformations
* Field extraction
* Formatting
* Conditional logic
* Metadata enrichment

RunnableLambda solves:

* “I need custom Python logic **inside** my chain”
* Without breaking LCEL composability

---

### Conceptual Difference

| Component           | Purpose                            |
| ------------------- | ---------------------------------- |
| RunnablePassthrough | Forward input unchanged            |
| **RunnableLambda**  | Transform input using custom logic |
| LLM / Retriever     | Heavy model or retrieval step      |

RunnableLambda = **glue logic**.

---

### Architecture View

![Image](https://cdn.analyticsvidhya.com/wp-content/uploads/2024/06/image-26-1.png)

![Image](https://miro.medium.com/v2/resize%3Afit%3A1200/1%2AOj7p9RbbH3BaeX1eZV3ebg.png)

![Image](https://miro.medium.com/1%2AX9hmWQYKaZevGO1SlGchGA.png)

---

## Demonstrations

---

### Basic RunnableLambda (Minimal)



In [1]:
from langchain_core.runnables import RunnableLambda

uppercase = RunnableLambda(lambda x: x.upper())

uppercase.invoke("langchain")


'LANGCHAIN'



**Output**

```
LANGCHAIN
```

Here:

* Input is transformed
* Output replaces input downstream

---

### RunnableLambda with Named Function (Preferred)



In [2]:
def word_count(text: str) -> int:
    return len(text.split())

count_runnable = RunnableLambda(word_count)

count_runnable.invoke("Runnable lambda is powerful")


4



**Output**

```
4
```

---

### RunnableLambda in a Chain



In [3]:
from langchain_openai import ChatOpenAI
from langchain_classic.prompts import ChatPromptTemplate

llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template(
    "Summarize this text in one sentence:\n{text}"
)

chain = (
    RunnableLambda(lambda x: x.strip())
    | RunnableLambda(lambda x: {"text": x})
    | prompt
    | llm
)

chain.invoke("LangChain enables composable AI pipelines")


AIMessage(content='LangChain allows for the creation and customization of AI pipelines that can be combined and integrated with each other.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 23, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CpyqZeJ9P8R3jx3zj1cZjJHLpfNeX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b4be1-f766-7e00-8731-2f5aebb3fe81-0', usage_metadata={'input_tokens': 23, 'output_tokens': 21, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})



What happens:

1. Trim whitespace
2. Wrap input into a dict
3. Prompt formatting
4. LLM call

---

### RunnableLambda for Field Extraction



In [4]:
extract_question = RunnableLambda(lambda x: x["question"])

extract_question.invoke(
    {"question": "What is RAG?", "user": "admin"}
)


'What is RAG?'



**Output**

```
What is RAG?
```

Common in:

* Tool outputs
* Agent state
* Structured pipelines

---

### RunnableLambda in RAG (Very Common)



In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# Create sample documents
docs = [
    Document(
        page_content="Retrieval Augmented Generation (RAG) is a technique that combines retrieval with generation.",
        metadata={"source": "doc1"}
    ),
    Document(
        page_content="RAG retrieves relevant documents from a knowledge base and uses them as context for the LLM.",
        metadata={"source": "doc2"}
    ),
    Document(
        page_content="LangChain provides tools to build RAG pipelines efficiently.",
        metadata={"source": "doc3"}
    )
]

# Create embeddings and vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

# Create retriever
retriever = vectorstore.as_retriever()

# Setup prompt template for RAG
prompt = ChatPromptTemplate.from_template(
    """
    Answer the question using the context.

    Question: {question}
    Context: {context}
    """
)

In [7]:
chain = (
    {
        "question": RunnableLambda(lambda x: x),
        "context": retriever
    }
    | prompt
    | llm
)



Here:

* RunnableLambda forwards/transforms input
* Retriever generates context
* Both are merged

(This is interchangeable with RunnablePassthrough when transformation is needed.)

---

### RunnableLambda with Conditional Logic



In [8]:
def normalize_query(q: str) -> str:
    if len(q) < 5:
        return f"Explain: {q}"
    return q

normalize = RunnableLambda(normalize_query)

normalize.invoke("RAG")


'Explain: RAG'



**Output**

```
Explain: RAG
```

---

### RunnableLambda for Metadata Enrichment



In [9]:
from datetime import datetime

add_metadata = RunnableLambda(
    lambda x: {
        "query": x,
        "timestamp": datetime.utcnow().isoformat()
    }
)

add_metadata.invoke("What is LangChain?")


C:\Users\sangouda\AppData\Local\Temp\ipykernel_40836\264983453.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat()


{'query': 'What is LangChain?', 'timestamp': '2025-12-23T15:47:43.199302'}



---

### RunnableLambda vs Plain Lambda

| Aspect              | RunnableLambda | Plain lambda |   |
| ------------------- | -------------- | ------------ | - |
| LCEL compatible     | ✅              | ❌            |   |
| Chainable with `    | `              | ✅            | ❌ |
| Async support       | ✅              | ❌            |   |
| Tracing / callbacks | ✅              | ❌            |   |
| Readability         | High           | Medium       |   |

---

### When to Use RunnableLambda

Use it when:

* Logic is simple and local
* You need transformation inside a chain
* You want LCEL composability
* You need tracing / observability

Avoid it when:

* Logic is complex → use a service or tool
* Heavy computation → use async workers

---

### Common Patterns

* Input normalization
* Output parsing
* Key mapping
* Guardrails
* Feature flags
* Routing decisions

---

### Key Takeaways

* RunnableLambda wraps **custom Python logic**
* It is the **transformation primitive** of LCEL
* Essential for real-world RAG and agent pipelines
* Best paired with RunnablePassthrough and retrievers
